## Manual Pipeline

In the previous notebook `0_simulation.ipynb` we simulated dataset. Our simulated dataset includes __detections__ and __resection_mni__. Now we are ready to run `Manual detections pipeline` using __detections__ as manually selected spikes and compare the outlut to the __resection_mni__ (ground truth). In this notebook we have the following goals:
1. Prepare and run irritative zone delineation using __detections__ timepoints.
2. Evaluate the quality of the irritative zone prediction using __resection__.

We will use the results presented in this nodebook as a baseline for the automated pipelines.

First, we import the simulation module and other packages. NOTE: To import the simulation we should change the working directory if we are running this example from the cloned GitHub repository.

In [1]:
import os
from pathlib import Path

import matplotlib.pylab as plt
import mne
import numpy as np

from nilearn import plotting
import nibabel as nb

# change to the root directory of the project
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

from megspikes.simulation.simulation import Simulation

# Setup the path for the simulation
sample_path = Path(os.getcwd()) / 'examples' / 'data'
sample_path.mkdir(exist_ok=True, parents=True)

%load_ext autoreload
%autoreload 2

/Users/valery/.local/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


## Prepare the simulation

In [4]:
sim = Simulation(sample_path)
sim.simulate_dataset()
sim

/Users/valery/MEG-SPIKES/megspikes/megspikes/casemanager/casemanager.py:152: UserWarning: No tsss fif file
  warnings.warn("No tsss fif file")


Simulation

Spike shapes file location: /Users/valery/MEG-SPIKES/megspikes/megspikes/simulation/data/spikes.npy
Simulated case parent folder: /Users/valery/MEG-SPIKES/megspikes/examples/data
Events ids: {'spike_shape_1': 1, 'spike_shape_2': 2, 'spike_shape_3': 3, 'spike_shape_4': 4}
Number of each event type: 15, 0, 0, 0
Events locations: G_temp_sup-G_T_transv-rh, G_temp_sup-G_T_transv-lh, S_subparietal-rh, S_subparietal-lh
Events activations: 120, 120, 120, 120

`sim.detections` is an array of spike peak localizations in samples. We will use it as manual detections.

In [6]:
sim.detections

array([  187,  1187,  2187,  3187,  4187,  5187,  6187,  7187,  8186,
        9187, 10187, 11187, 12187, 13187, 14187], dtype=int32)

`sim.clusters` is an array of clusters that corresponds to the simulation source index.

In [10]:
sim.clusters

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## Run Manual Pipeline

In [16]:
from megspikes.pipeline import manual_pipeline
from megspikes.database.database import read_meg_info_for_database
from sklearn import set_config
set_config(display='diagram')

In [13]:
db = read_meg_info_for_database(
    sim.case_manager.fif_file,
    sim.case_manager.fwd['ico5'])

In [17]:
pipe = manual_pipeline(sim.case_manager, db, 
                       sim.detections, sim.clusters - 1)
pipe

Pipeline(steps=[('make_clusters_library',
                 ManualDetections(clusters=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
                                  clusters_dataset=PosixPath('/Users/valery/MEG-SPIKES/megspikes/examples/data/sample/sample_alphacsc_cluster_results.nc'),
                                  database=<megspikes.database.database.Database object at 0x7fc7280a2d50>,
                                  detections=array([  187,  1187,  2187,  3187,  4187,  5187,  6187,  7187,  8186,
        9187, 10187, 11187, 12187, 13187, 14187], dtype=int32))),
                ('localize_clusters',
                 ClustersLocalization(case=<megspikes.casemanager.casemanager.CaseManager object at 0x7fc73f110e50>)),
                ('save_dataset',
                 SaveDataset(dataset=PosixPath('/Users/valery/MEG-SPIKES/megspikes/examples/data/sample/sample_manual_cluster_results.nc')))])

In [26]:
dataset, meg_data = pipe.fit_transform((None, sim.raw_simulation.copy()))

/Users/valery/MEG-SPIKES/megspikes/megspikes/database/database.py:491: UserWarning: spike.loc[{'detection_property': 'detection'}] values are all the same
  f"{da_name}.loc[{selection}] values are all the same")


`dataset` is an instance of xarray.Dataset, and it includes all the results. We can explore the results and the corresponding metadata.

In [38]:
dataset

<xarray.Dataset>
Dimensions:             (channel: 306, time: 14500, detection_property: 4, cluster: 1, cluster_property: 9, sensors: 2, source: 18840, time_evoked: 1000)
Coordinates:
  * channel             (channel) int64 0 1 2 3 4 5 ... 300 301 302 303 304 305
  * time                (time) float64 0.0 0.001 0.002 0.003 ... 14.5 14.5 14.5
  * detection_property  (detection_property) <U9 'detection' 'cluster' ... 'run'
  * cluster             (cluster) int64 0
  * cluster_property    (cluster_property) <U13 'cluster_id' ... 'time_peak'
  * sensors             (sensors) <U4 'grad' 'mag'
  * source              (source) int64 0 1 2 3 4 ... 18836 18837 18838 18839
  * time_evoked         (time_evoked) float64 0.0 0.001001 ... 0.999 1.0
Data variables:
    channel_names       (channel) <U8 'MEG 0113' 'MEG 0112' ... 'MEG 2641'
    spike               (time, detection_property) float64 0.0 0.0 ... 0.0 0.0
    cluster_properties  (cluster, cluster_property) float64 0.0 0.0 ... 499.0
    mne_localization    (sensors, cluster, source, time_evoked) float64 2.057...
    evoked              (cluster, channel, time_evoked) float64 1.55e-13 ... ...
Attributes:
    grad:     [  0   1   3   4   6   7   9  10  12  13  15  16  18  19  21  2...
    mag:      [  2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  4...

## View localized cluster

First we should convert numpy ndarray to mne.SourceEstimate. We use `array_to_stc` for this.

In [37]:
from megspikes.localization.localization import array_to_stc

stc = array_to_stc(dataset.mne_localization.values[0, 0, :, :],
                   sim.case_manager.fwd['ico5'],
                   sim.case_manager.case)

Now we can plot the cluster.

In [34]:
%matplotlib qt5
brain = stc.plot(subjects_dir=sim.case_manager.freesurfer_dir, hemi='both')